In [ ]:
import pandas as pd

In [ ]:
import networkx as nx
G=nx.DiGraph() 

#Creating the graph

with open("wiki-topcats-reduced.txt") as f:
    for line in f:
        list1 = line.strip().split("\t")
        G.add_node(list1[0])
        G.add_edge(list1[0],list1[1])

### adding attribute 'Category' to each node of the graph

for i in G:
    G.node[i]['Category']=[]
    
from collections import defaultdict
category_dict = defaultdict()

with open("wiki-topcats-categories.txt", "r") as f:
    lst2=G.nodes()
    for line in f:
        cat_list = line.strip().split(";")
        category = cat_list[0][9:]
        lst = cat_list[1].strip().split(" ")
        if len(lst) > 3500:   
            
            lst1=[el for el in lst if el in lst2]
            category_dict[category] = lst1

#Assign attributes to each node
for cat in category_dict:
    lst = category_dict[cat]
    for e in lst:
        if e in G:
            G.node[e]['Category'].append(cat)

In [3]:
dfg=pd.read_csv('Graphdf.csv')
dfg1 = pd.read_csv('Graphdf1.csv')
dfg2 = pd.read_csv('Graph2.csv')
dfg3 = pd.read_csv('Graph3.csv')

In [4]:
dfg = dfg.drop(columns="Unnamed: 0")
dfg1 = dfg1.drop(columns="Unnamed: 0")
dfg2 = dfg2.drop(columns="Unnamed: 0")
dfg3 = dfg3.drop(columns="Unnamed: 0")
dfg = pd.concat([dfg,dfg1,dfg2,dfg3])
dfg=dfg.reset_index(drop=True)
dfg = dfg.rename(columns={"0": "Category", "1": "Distance"})
dfg = dfg.sort_values(by='Distance')
dfg = dfg.reset_index(drop=True)
dfg

,Category,Distance
0,American_television_actors,6.000000
1,British_films,6.000000
2,English_television_actors,6.000000
3,Article_Feedback_Pilot,6.000000
4,English-language_films,6.000000
5,Debut_albums,7.000000
6,American_Jews,7.000000
7,English-language_albums,7.000000
8,Black-and-white_films,7.000000
9,People_from_New_York_City,7.000000


Last point: We want to keep an information, score, into each node. first, we initialized every node as string, in order to make a double check (if we still didn't pass throught that node it will be still a string, but if there are no in edges this information will be 0). This initialization will be done into G, our original graph.

In [5]:
nodes_G = G.nodes()
for n in nodes_G:
    G.node[n]['score'] = ''

In [6]:
input_category='Indian_films'
c0 = category_dict[input_category][0:2000]    
sub = G.subgraph(c0)
for s in sub:
    G.node[s]['score'] = len(sub.in_edges(s))

In [7]:
categories = dfg['Category'].values #taking all categories from the ranking dataframe 

In [8]:
l = c0 #list of nodes in the subgraph (now only input_category nodes)
for c in categories:
    l = l + category_dict[c]
    sub1 = G.subgraph(l)
    for e1 in sub1: 
        if not isinstance(G.node[e1]['score'],str):
            continue
        else:
            ie1 = sub1.in_edges(e1)
            ie1 = [el[0] for el in ie1]
                   
            score = 0
            for f in ie1:
                
                if G.node[f]['score'] == '':
                    score +=1
                else:
                    score += G.node[f]['score']
            G.node[e1]['score'] = score
    

In [ ]:
DfGraph=pd.DataFrame(columns=['Node','Score'])
i=0
for e in G:
    if not isinstance(G.node[e]['score'],str):
        DfGraph.loc[i,'Node']=e
        DfGraph.loc[i,'Score']=G.node[e]['score']
        i+=1

In [19]:
from numba import autojit, prange
list1 = []

@autojit
def graphscore(e):
    score_dict={}
    score_dict['Node']=e
    score_dict['Score']=G.node[e]['score']
    return(score_dict)
for e in G:
    list1.append(graphscore(e))

In [28]:
dfscore=pd.DataFrame(list1)

In [33]:
dfscore.head(100)

,Node,Score
0,52,0
1,401135,25602441078634801288154
2,1069112,352627517790
3,1163551,795839622449460363564225
4,62,0
5,12162,4776532295377016330911836413
6,167659,0
7,279122,9270010809
8,1089199,189867578797
9,1354553,0
